Basket testing with layer1 coins, trading on btc

In [2]:
from dotenv import load_dotenv
from binance.client import Client as bnb_client
import os

load_dotenv()
# need TLD in US if US
client = bnb_client(tld='US', api_key=os.getenv('BINANCE_API'), api_secret=os.getenv('BINANCE_SECRET'))

In [3]:
all_binance_symbols =[symbol['symbol'] for symbol in client.get_exchange_info()['symbols']]
print(len(all_binance_symbols))

all_usdt_pairs = [symbol['symbol']  for symbol in client.get_exchange_info()['symbols'] if symbol['quoteAsset'] == 'USDT']
len(all_usdt_pairs)

547


176

In [4]:

import datetime
import pandas as pd 
import numpy as np 

def get_binance_px(symbol,freq,start_ts = '2020-01-01'):
    data = client.get_historical_klines(symbol,freq,start_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    data['open_time'] = data['open_time'].map(lambda x: datetime.datetime.fromtimestamp(x/1000, datetime.timezone.utc))
    data['close_time'] = data['close_time'].map(lambda x: datetime.datetime.fromtimestamp(x/1000, datetime.timezone.utc))
    return data 

univ = all_binance_symbols

freq = '1h'
filename = 'binance_{freq}_px.pkl'.format(freq=freq)
if filename not in os.listdir():
    px = {}
    for x in univ:
        data = get_binance_px(x,freq)
        px[x] = data.set_index('open_time')['close']

    px = pd.DataFrame(px).astype(float)
    px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq=freq))
    px.to_pickle(filename)
else:
    px = pd.read_pickle(filename)
ret = px.pct_change()

C:\Users\joshu\AppData\Local\Temp\ipykernel_40584\366739778.py:32: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = px.pct_change()


In [5]:
import requests

def get_layer1_coin_names():
    # Step 1: Define the API endpoint and parameters
    api_url = f'https://api.coingecko.com/api/v3/coins/markets?api_key={os.getenv('COINGECKO_API_KEY2')}'
    category = 'smart-contract-platform'  # Adjust based on actual category
    per_page = 250
    page = 1
    vs_currency = 'usd'
    order = 'market_cap_desc'
    sparkline = 'false'

    coin_names = []
    coin_ticker = []

    while True:
        params = {
            'vs_currency': vs_currency,
            'category': category,
            'order': order,
            'per_page': per_page,
            'page': page,
            'sparkline': sparkline
        }

        response = requests.get(api_url, params=params)

        if response.status_code != 200:
            print(f"Error: Unable to fetch data (Status Code: {response.status_code})")
            break

        data = response.json()

        if not data:
            # No more data to fetch
            break
        
        # Extract coin names
        for coin in data:
            coin_names.append(coin['name'])
            coin_ticker.append(coin['symbol'])

        print(f"Fetched page {page} with {len(data)} coins.")

        page += 1
    return coin_ticker

layer1_coins = get_layer1_coin_names()
print(f"\nTotal Layer 1 Coins Found: {len(layer1_coins)}")


Fetched page 1 with 250 coins.
Fetched page 2 with 250 coins.
Fetched page 3 with 213 coins.

Total Layer 1 Coins Found: 713


In [17]:
exchange_info = client.get_exchange_info()
binance_base_assets = [x['baseAsset'] for x in exchange_info['symbols']]

In [7]:
layer1_coin = [x.upper() for x in layer1_coins]
layer1_coin = [x for x in layer1_coin if x in binance_base_assets]
len(layer1_coin)

82

In [30]:
binance_layer1_symbols = [x['symbol'] for x in exchange_info['symbols'] if (x['baseAsset'] in layer1_coin and x['quoteAsset'] == 'USDT')]
layer1_binance_ret = ret[binance_layer1_symbols]

In [31]:
ret

,BTCUSD4,ETHUSD4,XRPUSD,BCHUSD4,LTCUSD4,USDTUSD4,BTCUSDT,ETHUSDT,XRPUSDT,BCHUSDT,...,FORTUSDT,SUIUSDT,ONGUSDT,GUSDT,RENDERUSDT,BONKUSDT,MAGICUSDT,PEPEUSDT,WIFUSDT,IOTXUSDT
2020-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 01:00:00+00:00,0.004347,0.010273,0.005729,0.012230,0.008268,-0.000701,0.005123,0.010861,0.005089,0.015264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 02:00:00+00:00,0.004049,0.004776,0.003625,0.000243,0.000724,0.000000,0.003556,0.004682,0.004340,0.001644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 03:00:00+00:00,-0.002601,-0.006057,-0.002064,-0.005045,0.000482,-0.000201,-0.001011,-0.005576,-0.001749,-0.007529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 04:00:00+00:00,-0.000952,0.000231,-0.001551,0.001999,0.000241,-0.000201,-0.002034,0.000845,0.000000,0.004620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-03 18:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003785,0.007191,0.002171,0.011054,...,0.006014,0.008938,0.000000,0.009718,0.010758,0.008094,0.000000,0.002792,0.000000,0.002798
2025-01-03 19:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001128,0.003760,0.011814,-0.007149,...,0.001708,0.024046,0.000321,0.000000,-0.000124,-0.021872,0.000000,0.000928,-0.014925,0.000000
2025-01-03 20:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001800,-0.003807,-0.009697,-0.004871,...,0.006820,-0.003419,0.000000,-0.006209,-0.010769,0.002830,-0.092425,-0.016226,-0.010560,0.004721
2025-01-03 21:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002056,0.001304,-0.002774,0.000000,...,0.000847,0.006696,0.000000,0.000000,-0.000876,0.007056,0.000000,0.016494,-0.000928,0.000000


Signal

Each hour, buy losing coins within the industry, and sell a basket of the most correlated in industry above 0.3 with equal weighting


or do the opposite
 parameters 
 - number of coins to buy/sell without basket
 - buy winner or loser?
 - basket weighitngs/choosing parameters (what stocks, weightings of basket portfolio)
 - time horizon
 - when to trade
